In [1]:
import pandas as pd
datapath = "audioset/"
train_df = pd.read_csv(datapath+"train.csv", sep="\t")
dev_df = pd.read_csv(datapath+"dev.csv", sep="\t")
eval_df = pd.read_csv(datapath+"eval.csv", sep="\t")

In [25]:
dev_df.head()

,filename,scene_label
0,dev/airport-lisbon-1000-41312-a.wav,airport
1,dev/airport-lisbon-1000-41323-a.wav,airport
2,dev/airport-lisbon-1122-41394-a.wav,airport
3,dev/airport-lisbon-1114-41409-a.wav,airport
4,dev/airport-prague-1069-40978-a.wav,airport


In [2]:
file_list = list(dev_df.filename)
label_list = list(dev_df.scene_label)

In [3]:
from Extract_feature import extract_mfcc
feature_list = []
for filename in file_list:
    wav_file_path = datapath+filename
    mfcc_feature = extract_mfcc(wav_file_path, option="fbanks")
    feature_list.append(mfcc_feature)           # dev: 17s

In [4]:
feature_list[1].shape

(499, 26)

In [9]:
wav0_feature = feature_list[1]

In [11]:
wav0_feature

array([[ 0.48511033,  0.39850428,  0.6371938 , ..., -0.75727791,
        -0.93305286, -1.08134898],
       [ 0.37283944,  0.30500768,  0.40041704, ..., -0.76556033,
        -0.8548776 , -0.96588157],
       [ 0.32927345,  0.26694104,  0.28341197, ..., -0.7846863 ,
        -0.93070274, -1.00495668],
       ...,
       [ 0.15812107,  0.20399878,  0.26828782, ..., -0.82023812,
        -1.02665997, -1.09487666],
       [ 0.30973284,  0.18837069,  0.03434392, ..., -0.6862292 ,
        -0.91447668, -1.05280664],
       [ 0.37761239,  0.44850579,  0.17752808, ..., -0.6754242 ,
        -0.86746738, -0.93432526]])

In [15]:
wav0_feature.flatten()[0:30]

array([ 0.48511033,  0.39850428,  0.6371938 ,  0.42808497,  0.26575998,
        0.19421591,  0.13600567,  0.11473331,  0.08688659,  0.09836886,
       -0.03113776,  0.04360556,  0.1933422 ,  0.0148299 , -0.28232775,
       -0.36112402, -0.51184785, -0.62078813, -0.91989912, -0.70234364,
       -0.54192293, -0.5485156 , -0.65183053, -0.75727791, -0.93305286,
       -1.08134898,  0.37283944,  0.30500768,  0.40041704,  0.23974215])

In [18]:
wav0_feature = wav0_feature.reshape(1,-1)
wav0_feature[:,:30]

array([[ 0.48511033,  0.39850428,  0.6371938 ,  0.42808497,  0.26575998,
         0.19421591,  0.13600567,  0.11473331,  0.08688659,  0.09836886,
        -0.03113776,  0.04360556,  0.1933422 ,  0.0148299 , -0.28232775,
        -0.36112402, -0.51184785, -0.62078813, -0.91989912, -0.70234364,
        -0.54192293, -0.5485156 , -0.65183053, -0.75727791, -0.93305286,
        -1.08134898,  0.37283944,  0.30500768,  0.40041704,  0.23974215]])

In [19]:
set(label_list)

{'airport',
 'bus',
 'metro',
 'metro_station',
 'park',
 'public_square',
 'shopping_mall',
 'street_pedestrian',
 'street_traffic',
 'tram'}

In [28]:
target_df = train_df[train_df.scene_label=='metro']
len(target_df)

296

In [29]:
target_df

,filename,scene_label
599,train/metro-prague-1016-40020-a.wav,metro
600,train/metro-prague-1022-40026-a.wav,metro
601,train/metro-prague-1026-40030-a.wav,metro
602,train/metro-prague-1054-40076-a.wav,metro
603,train/metro-prague-1026-40088-a.wav,metro
...,...,...
890,train/metro-milan-1142-41083-a.wav,metro
891,train/metro-milan-1142-41093-a.wav,metro
892,train/metro-milan-1025-41132-a.wav,metro
893,train/metro-milan-1025-41135-a.wav,metro


In [26]:
from Extract_feature import extract_mfcc

labels = ['airport', 'bus', 'metro', 'metro_station',
          'park', 'public_square', 'shopping_mall',
          'street_pedestrian', 'street_traffic', 'tram']
sum_num=0
for c in labels:
    target_df = train_df[train_df.scene_label==c]
    sum_num+=len(target_df)
print(sum_num)

3008


In [30]:
import numpy as np
# prepare data for each class, corresponding to GMM
for c in labels:
    target_df = train_df[train_df.scene_label==c]
    file_list = list(target_df.filename)
    feature_list = []
    for filename in file_list:
        wav_file_path = datapath+filename
        mfcc_feature = extract_mfcc(wav_file_path, option="mfcc")
        feature_list.append(mfcc_feature.reshape(1,-1))         # [1, frames*n_features]
    feature_list = np.concatenate(feature_list, axis=0)
    print(feature_list.shape)
# 80s

(300, 6487)
(299, 6487)
(296, 6487)
(296, 6487)
(312, 6487)
(312, 6487)
(296, 6487)
(296, 6487)
(307, 6487)
(294, 6487)


In [116]:
import numpy as np
# prepare data for each class, corresponding to GMM
labels = ['airport', 'bus', 'metro', 'metro_station',
          'park', 'public_square', 'shopping_mall',
          'street_pedestrian', 'street_traffic', 'tram']
target_df = dev_df[dev_df.scene_label==labels[1]]
file_list = list(target_df.filename)
feature_list = []
for filename in file_list:
    wav_file_path = datapath+filename
    mfcc_feature = extract_mfcc(wav_file_path, option="mfcc")
    feature_list.append(mfcc_feature.reshape(1,-1))         # [1, frames*n_features]
feature_list = np.concatenate(feature_list, axis=0)
print(feature_list.shape)
# 10s

(40, 6487)


Kmeans

In [34]:
from sklearn.cluster import KMeans

file_list2 = list(dev_df.filename)
label_list2 = list(dev_df.scene_label)

feature_list2 = []
for filename in file_list2:
    wav_file_path = datapath+filename
    mfcc_feature = extract_mfcc(wav_file_path, option="mfcc")
    feature_list2.append(mfcc_feature.reshape(1,-1))         # [1, frames*n_features]
feature_list2 = np.concatenate(feature_list2, axis=0)
print(feature_list2.shape)

(379, 6487)


In [72]:
model_km = KMeans(n_clusters=10, max_iter=100, random_state=0)
model_km.fit(feature_list2)

e:\Programming_Language\Python\anaconda\ANACONDA3_2021_11\envs\pytorch_170\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


KMeans(max_iter=8, n_clusters=10, random_state=0)

In [57]:
model_km.cluster_centers_.shape

(10, 6487)

In [82]:
from collections import Counter
Counter(model_km.labels_)

Counter({5: 20, 3: 19, 7: 53, 9: 48, 0: 45, 4: 63, 6: 45, 1: 27, 2: 23, 8: 36})

In [79]:
Counter(model_km.labels_[37:37+40])

Counter({9: 7, 1: 1, 4: 7, 7: 2, 6: 8, 0: 1, 8: 9, 2: 5})

In [80]:
start, end = 0, 0
for c in labels:
    target_df = dev_df[dev_df.scene_label==c]
    end += len(target_df)
    print(Counter(model_km.labels_[start:end]))
    start = end

Counter({9: 11, 6: 9, 0: 4, 7: 3, 4: 3, 2: 2, 8: 2, 5: 1, 3: 1, 1: 1})
Counter({8: 9, 6: 8, 9: 7, 4: 7, 2: 5, 7: 2, 1: 1, 0: 1})
Counter({0: 11, 5: 8, 6: 5, 4: 3, 7: 3, 9: 3, 3: 2, 2: 1, 1: 1})
Counter({7: 8, 0: 8, 4: 5, 1: 4, 9: 4, 3: 3, 5: 2, 6: 2, 8: 1})
Counter({8: 12, 2: 11, 4: 10, 0: 3, 1: 1, 6: 1, 9: 1})
Counter({4: 8, 1: 8, 9: 5, 0: 3, 6: 3, 5: 3, 7: 3, 8: 3, 3: 2, 2: 1})
Counter({7: 15, 4: 6, 6: 5, 1: 4, 5: 3, 0: 2, 9: 1})
Counter({4: 11, 7: 8, 9: 8, 6: 7, 8: 2, 0: 1, 1: 1})
Counter({3: 11, 7: 9, 0: 7, 5: 3, 6: 3, 1: 3, 4: 2, 8: 1})
Counter({9: 8, 4: 8, 8: 6, 0: 5, 1: 3, 2: 3, 7: 2, 6: 2})


In [55]:
model_km.labels_[:30]

array([5, 3, 7, 7, 9, 9, 0, 4, 6, 6, 9, 9, 1, 2, 0, 9, 6, 9, 4, 6, 6, 9,
       6, 9, 6, 9, 6, 2, 8, 8])

In [ ]:
model_km = KMeans(n_clusters=10, max_iter=100, random_state=0)
model_km.fit(feature_list2)

In [114]:
import numpy as np
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=4, max_iter=100, n_init=3, tol=1e-3,
                        covariance_type='diag', init_params="kmeans", random_state=0)
gmm.fit(feature_list)

e:\Programming_Language\Python\anaconda\ANACONDA3_2021_11\envs\pytorch_170\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
e:\Programming_Language\Python\anaconda\ANACONDA3_2021_11\envs\pytorch_170\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
e:\Programming_Language\Python\anaconda\ANACONDA3_2021_11\envs\pytorch_170\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  w

GaussianMixture(covariance_type='diag', n_components=4, n_init=3,
                random_state=0)

In [93]:
gmm.score_samples(feature_list)     # diag

array([ 5.32794999e+02,  1.62780513e+03,  3.15230981e+02,  1.25591652e+01,
        3.19337590e+03,  2.65292686e+03,  9.88253447e+02,  3.17798726e+03,
        1.38447167e+03,  1.92427704e+03,  8.61382467e+02, -3.74409423e+02,
        1.64103780e+02,  1.26220855e+03,  3.88458433e+04,  1.71267677e+01,
        3.49877776e+03,  1.84201064e+03,  1.88959868e+03,  2.96626776e+03,
       -1.04237378e+02, -3.11394915e+03,  9.34829890e+01,  1.26732850e+03,
       -3.27186523e+02,  2.00209263e+03,  3.55090068e+03,  3.00873864e+02,
        2.78900548e+02,  3.29500827e+03,  1.39145694e+03,  2.27078002e+03,
        6.27621488e+02,  1.99198121e+02,  2.02189797e+03,  1.06286573e+03,
        1.23409626e+03])

In [117]:
gmm.score_samples(feature_list)     # diag

array([  -260.64910166,  -2416.64705399,  -3618.62331995,  -5452.99321695,
        -2531.92593425,  -2472.78313564,  -2456.10495518,  -1414.05722195,
         -222.61008912,  -1128.69884498,   -698.15872574,   1052.41195214,
        -4011.95618982,  -2210.35396385,  -3615.91767248,  -6000.8715427 ,
        -8820.72420835, -11007.47882881, -14194.02299791, -13048.20043298,
        -5090.3120214 ,  -4598.44024807,  -3370.20222072,  -4934.30863766,
        -3822.17143657,  -3371.04056275,  -2085.31879945,  -3998.58694596,
        -8656.64656366,  -7051.99101229,  -7162.31110995,  -8839.68265636,
        -3938.63302175,  -2085.13337172,  -2583.61083444,  -3750.34290608,
        -3913.70594591,  -3301.63850557,  -2889.42102516,  -2897.27804954])

In [113]:
gmm.score_samples(feature_list)     # full

array([-3.70668936e+08, -5.05518193e+08, -2.22132518e+08, -1.83398790e+08,
       -2.32698660e+08, -2.12200226e+08, -2.69168171e+08, -2.48161124e+08,
       -5.73360852e+08, -2.07134394e+08, -5.05350309e+08, -2.37989197e+08,
       -4.20301152e+08, -2.73833877e+08, -2.26384377e+08, -2.62606722e+08,
       -4.97260918e+08, -3.43513719e+08, -2.60299297e+08, -2.76206377e+08,
       -2.97192167e+08, -2.76350811e+08, -2.28203676e+08, -2.13915802e+08,
       -2.93681582e+08, -1.98260077e+08, -2.80487282e+08, -4.98356430e+08,
       -4.59960572e+08, -3.47080106e+08, -2.38439313e+08, -2.87311377e+08,
       -5.90868227e+08, -2.55206735e+08, -4.55887645e+08, -4.96737087e+08,
       -2.27070653e+08])

In [110]:
gmm.n_iter_

2

In [111]:
gmm.covariances_.shape      # diag

(4, 6487, 6487)

----

In [ ]:
# train
from sklearn.cluster import KMeans

train_epochs = 1
model_km = KMeans(n_clusters=10, max_iter=50, random_state=0)
for _ in range(train_epochs):
    model_km = model_km.fit(train_features)
    print(model_km.labels_[:10])